In [41]:
import torch

torch.manual_seed(0)
import numpy as np

In [42]:
B, J, N = 1, 2, 5  # batch size, betas, agents

# Simulate typical network outputs

w = torch.rand(B, J, N)
A_diag = torch.rand(B, J, N) + 0.1  # avoid zeros to stay positive definite
b = torch.randn(B, J, N)
c = torch.randn(B, J)

In [43]:
w * A_diag

tensor([[[0.2228, 0.3854, 0.0108, 0.0355, 0.1211],
         [0.3922, 0.3909, 0.8068, 0.1189, 0.2417]]])

In [44]:
(w * (A_diag * w))

tensor([[[0.1105, 0.2961, 0.0010, 0.0047, 0.0372],
         [0.2487, 0.1916, 0.7233, 0.0542, 0.1528]]])

In [45]:
# sum of Ai * wi^2
(A_diag * w**2).sum(dim=2)

tensor([[0.4495, 1.3706]])

In [46]:
print(A_diag.shape)

torch.Size([1, 2, 5])


In [47]:
quad_terms_manual = torch.zeros((B, J))

quad_sum = 0
for i in range(B):
    for j in range(J):
        quad_sum = 0.0
        for k in range(N):
            quad_sum += A_diag[i, j, k].item() * (w[i, j, k].item() ** 2)
        quad_terms_manual[i, j] = quad_sum

quad_terms_manual

tensor([[0.4495, 1.3706]])

In [48]:
(A_diag * w.pow(2)).sum(dim=2)

tensor([[0.4495, 1.3706]])

In [51]:
import os, sys


def get_dir_n_levels_up(path, n):
    # Go up n levels from the given path
    for _ in range(n):
        path = os.path.dirname(path)
    return path


proj_root = get_dir_n_levels_up(os.path.abspath("__file__"), 3)
sys.path.append(proj_root)

from opinion_dynamics.models import OpinionNet

B, J, N = 2, 2, 3
model = OpinionNet(nr_agents=N, nr_betas=J)
model.train()

print(model.predict_A_b_c.weight.grad) 

x = torch.randn(B, N)
out = model(x)

loss = out["c"].sum()
loss.backward()

model.predict_A_b_c.weight.grad 

None


tensor([[0.0000, 0.4039, 0.7672, 0.3739, 0.0000, 0.1849, 0.9487, 0.0000, 0.2697,
         0.0934, 0.6150, 0.0000, 0.0000, 0.7801, 0.5131, 0.0000, 0.0000, 0.0000,
         0.0000, 0.3994, 0.6054, 0.0000, 0.7346, 0.0000, 0.1254, 0.7723, 0.0000,
         0.8807, 0.1740, 0.0210, 0.0000, 0.1842, 0.9484, 0.2511, 0.0000, 0.4105,
         0.4136, 0.1318, 0.5598, 0.0000, 0.0000, 0.1904, 0.8897, 0.1078, 0.0000,
         0.0000, 0.0532, 0.0000, 0.6144, 0.0000, 0.0000, 0.0000, 0.1475, 0.0000,
         0.1591, 0.0000, 0.4176, 0.0000, 0.2064, 0.1283, 0.3935, 0.4327, 0.0000,
         0.2422],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0

In [50]:
model.predict_A_b_c.weight.shape

torch.Size([27, 64])